# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,19.08,72,88,9.97,AU,1717172415
1,1,port-aux-francais,-49.3500,70.2167,6.39,95,100,15.51,TF,1717172412
2,2,souillac,-20.5167,57.5167,24.21,69,40,5.14,MU,1717172510
3,3,cabedelo,-6.9811,-34.8339,29.37,72,58,6.77,BR,1717172510
4,4,vorgashor,67.5833,63.9500,-2.33,87,100,5.54,RU,1717172510


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 25 degrees but higher than 20
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 20)]

# Wind speed less than 3.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 3.5)]

# Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,bethel,41.3712,-73.4140,22.97,45,0,0.45,US,1717171998
35,35,changji,44.0167,87.3167,22.44,33,0,1.00,CN,1717172514
106,106,ilhabela,-23.7781,-45.3581,21.28,71,0,1.17,BR,1717172524
206,206,toliara,-23.3500,43.6667,22.77,67,0,3.34,MG,1717172560
228,228,lucas,41.6501,-83.6666,23.07,31,0,2.57,US,1717172563
258,258,flagstaff,35.1981,-111.6513,23.64,21,0,0.89,US,1717172566
358,358,buka,40.8108,69.1986,23.95,26,0,2.41,UZ,1717172580
391,391,feodosiya,45.0333,35.3833,21.49,62,0,2.93,UA,1717172585
476,476,livingstone,-17.8419,25.8543,21.75,29,0,0.24,ZM,1717172241
485,485,dois vizinhos,-25.7336,-53.0572,20.20,44,0,1.86,BR,1717172366


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,bethel,US,41.3712,-73.4140,45,
35,changji,CN,44.0167,87.3167,33,
106,ilhabela,BR,-23.7781,-45.3581,71,
206,toliara,MG,-23.3500,43.6667,67,
228,lucas,US,41.6501,-83.6666,31,
258,flagstaff,US,35.1981,-111.6513,21,
358,buka,UZ,40.8108,69.1986,26,
391,feodosiya,UA,45.0333,35.3833,62,
476,livingstone,ZM,-17.8419,25.8543,29,
485,dois vizinhos,BR,-25.7336,-53.0572,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel (accommodation.hotel from the Geocoding API)
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
changji - nearest hotel: No hotel found
ilhabela - nearest hotel: Vila Kebaya
toliara - nearest hotel: Ambary
lucas - nearest hotel: Hawthorn Suites by Wyndham Holland/Toledo Area
flagstaff - nearest hotel: Residence Inn
buka - nearest hotel: No hotel found
feodosiya - nearest hotel: Afrodita
livingstone - nearest hotel: New Fairmount Hotel and Casino
dois vizinhos - nearest hotel: Hotel Requinte


,City,Country,Lat,Lng,Humidity,Hotel Name
25,bethel,US,41.3712,-73.4140,45,Hampton Inn Danbury
35,changji,CN,44.0167,87.3167,33,No hotel found
106,ilhabela,BR,-23.7781,-45.3581,71,Vila Kebaya
206,toliara,MG,-23.3500,43.6667,67,Ambary
228,lucas,US,41.6501,-83.6666,31,Hawthorn Suites by Wyndham Holland/Toledo Area
258,flagstaff,US,35.1981,-111.6513,21,Residence Inn
358,buka,UZ,40.8108,69.1986,26,No hotel found
391,feodosiya,UA,45.0333,35.3833,62,Afrodita
476,livingstone,ZM,-17.8419,25.8543,29,New Fairmount Hotel and Casino
485,dois vizinhos,BR,-25.7336,-53.0572,44,Hotel Requinte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)